### Import dependencies

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
#!pip install squarify
#plt.style.use('fivethirtyeight')
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.tools as tls
import pygsheets
%matplotlib inline


### Authorize connection between Jupyter notebook (python working environment; like R studio) and google sheets so we can run analyses as the sheet is updated

In [2]:
gc = pygsheets.authorize() 
# Use customized credentials 
gc = pygsheets.authorize(client_secret='client_secret.json')
# For the first time, it will may produce as a link to authorize

### Open spreadsheet by name

In [3]:
sh = gc.open('PythonCurrentFlow_Aging')

### Open "Data" worksheet (can also use sh.sheet1)

In [4]:
data = sh[0]

### Get worksheet values as pandas dataframe

In [5]:
aging_data = pd.DataFrame(data.get_all_records())

In [6]:
aging_data

,Genus,Species,Sample ID,Species Code,Site,River,Basin,Lat,Long,Date Collected,...,Status Upon Collection,Final Age,Age dif,Z age,L age,B age,K&M Age,K age,M age,Notes
0,Amblema,plicata,1,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,15,,,,,15,15,14,
1,Amblema,plicata,2,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,16,,,,,16,16,12,
2,Amblema,plicata,3,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,17,,,,,17,17,9,
3,Amblema,plicata,4,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,15,,,,,15,13,12,
4,Amblema,plicata,5,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,14,,,,,14,14,12,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,Lampsilis,teres,471,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,
470,Lampsilis,teres,472,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,
471,Lampsilis,teres,473,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,
472,Lampsilis,teres,474,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,B: how could you tell difference b/w 0.5 & 1?


#### Rename columns 

In [7]:
df_rename=aging_data.rename(columns={"Sample ID": "id", "Genus":"genus", "Species":"species", "Species Code": "code", "Site":"site", "River":"river", "Basin":"basin", "Lat":"lat", "Long":"long", "Date Collected":"date", "Status Upon Collection":"status", "Ager":"ager", "Age":"age"})
df_rename

,genus,species,id,code,site,river,basin,lat,long,date,...,status,Final Age,Age dif,Z age,L age,B age,K&M Age,K age,M age,Notes
0,Amblema,plicata,1,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,15,,,,,15,15,14,
1,Amblema,plicata,2,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,16,,,,,16,16,12,
2,Amblema,plicata,3,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,17,,,,,17,17,9,
3,Amblema,plicata,4,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,15,,,,,15,13,12,
4,Amblema,plicata,5,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,14,,,,,14,14,12,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,Lampsilis,teres,471,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,
470,Lampsilis,teres,472,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,
471,Lampsilis,teres,473,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,
472,Lampsilis,teres,474,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,B: how could you tell difference b/w 0.5 & 1?


### What do age measurements for L. teres look like across Z, L & B?
#### Compare value counts 

In [11]:
pd.crosstab(index=aging_data['Species']=='teres', columns=aging_data['Z age'])

Z age,0.5,1,2,3,4,5,6,7,8,9,12,13,,NA
Species,,,,,,,,,,,,,,
False,0,1,1,0,0,0,0,0,1,0,1,1,320,0
True,7,6,12,33,22,18,14,10,7,2,0,0,16,2


##### L. teres
#### Ager Z: aged most mussels at 3 yrs old

In [110]:
pd.crosstab(index=aging_data['Species']=='teres', columns=aging_data['L age'])

L age,0.5,1,2,3,4,5,6,7,8,9,10,11,13,
Species,,,,,,,,,,,,,,
plicata,0,1,1,0,0,0,0,0,0,1,1,0,1,320
teres,7,0,6,19,26,34,17,8,10,1,2,1,0,18


##### L. teres
#### Ager L: aged most mussels at 5 yrs old

In [10]:
pd.crosstab(index=aging_data['Species']=='teres', columns=aging_data['B age'])

B age,1,2,3,4,5,6,7,8,9,,,
Species,,,,,,,,,,,,
False,0,0,0,0,0,0,0,0,0,325,0,0
True,9,12,25,27,21,12,10,5,2,24,1,1


##### L. teres
#### Ager B: aged most mussels at 4 yrs old

In [12]:
pd.crosstab(index=aging_data['Species']=='teres', columns=aging_data['Site'])

Site,"Altair, TX","Bay City, TX","Bellville, TX","Gonzales, TX","Navasota, TX","Simonton, TX","Victoria, TX"
Species,,,,,,,
False,100,0,5,101,19,0,100
True,13,67,19,0,0,50,0


### Take a look at data (.head shows first 5 rows)

In [82]:
df_rename.head()

,genus,species,id,code,site,river,basin,lat,long,date,...,status,Final Age,Age dif,Z age,L age,B age,K&M Age,K age,M age,Notes
0,Amblema,plicata,1,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,15,,,,,15,15,14,
1,Amblema,plicata,2,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,16,,,,,16,16,12,
2,Amblema,plicata,3,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,17,,,,,17,17,9,
3,Amblema,plicata,4,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,15,,,,,15,13,12,
4,Amblema,plicata,5,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,14,,,,,14,14,12,


## Clean data
### Drop some columns

In [15]:
drop_df=df_rename.drop(columns=['Final Age', 'Age dif', 'K&M Age', 'K age', 'M age', 'Notes'])
drop_df.head()

,genus,species,id,code,site,river,basin,lat,long,date,Length (mm),status,Z age,L age,B age
0,Amblema,plicata,1,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,74.0,Alive,,,
1,Amblema,plicata,2,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,72.0,Alive,,,
2,Amblema,plicata,3,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,76.0,Alive,,,
3,Amblema,plicata,4,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,76.0,Alive,,,
4,Amblema,plicata,5,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,77.0,Alive,,,


### Select teres data (b/c X, L and B have aged)

In [29]:
teres_df = drop_df.loc[drop_df["species"] == "teres"]
teres_df

,genus,species,id,code,site,river,basin,lat,long,date,Length (mm),status,Z age,L age,B age
295,Lampsilis,teres,297,E,"Altair, TX",Colorado,Colorado,29.595299°,-96.453229°,10/3/19,128.0,Alive,6,6,
296,Lampsilis,teres,298,E,"Altair, TX",Colorado,Colorado,29.595299°,-96.453229°,10/3/19,95.0,Alive,,,
297,Lampsilis,teres,299,E,"Altair, TX",Colorado,Colorado,29.595299°,-96.453229°,10/3/19,81.0,Alive,,,
298,Lampsilis,teres,300,E,"Altair, TX",Colorado,Colorado,29.595299°,-96.453229°,10/3/19,125.0,Alive,5,5,
299,Lampsilis,teres,301,E,"Altair, TX",Colorado,Colorado,29.595299°,-96.453229°,10/3/19,115.0,Alive,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,Lampsilis,teres,471,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,32.5,Alive,0.5,0.5,1
470,Lampsilis,teres,472,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,35.5,Alive,0.5,0.5,1
471,Lampsilis,teres,473,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,33.0,Alive,0.5,0.5,1
472,Lampsilis,teres,474,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,19.0,Alive,0.5,0.5,1


In [28]:
teres_df['site'].value_counts()

Bay City, TX     67
Simonton, TX     50
Bellville, TX    19
Altair, TX       13
Name: site, dtype: int64

In [30]:
teres_df['species'].value_counts()

teres    149
Name: species, dtype: int64

In [31]:
teres_df.head()

,genus,species,id,code,site,river,basin,lat,long,date,Length (mm),status,Z age,L age,B age
295,Lampsilis,teres,297,E,"Altair, TX",Colorado,Colorado,29.595299°,-96.453229°,10/3/19,128.0,Alive,6,6,
296,Lampsilis,teres,298,E,"Altair, TX",Colorado,Colorado,29.595299°,-96.453229°,10/3/19,95.0,Alive,,,
297,Lampsilis,teres,299,E,"Altair, TX",Colorado,Colorado,29.595299°,-96.453229°,10/3/19,81.0,Alive,,,
298,Lampsilis,teres,300,E,"Altair, TX",Colorado,Colorado,29.595299°,-96.453229°,10/3/19,125.0,Alive,5,5,
299,Lampsilis,teres,301,E,"Altair, TX",Colorado,Colorado,29.595299°,-96.453229°,10/3/19,115.0,Alive,,,


### Download dependencies for figures

In [22]:
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.figure_factory as ff

### Valid properties for plotly.graph_objects (go)

In [ ]:
#  Valid properties:
#         activeshape
#             :class:`plotly.graph_objects.layout.Activeshape`
#             instance or dict with compatible properties
#         angularaxis
#             :class:`plotly.graph_objects.layout.AngularAxis`
#             instance or dict with compatible properties
#         annotations
#             A tuple of
#             :class:`plotly.graph_objects.layout.Annotation`
#             instances or dicts with compatible properties
#         annotationdefaults
#             When used in a template (as
#             layout.template.layout.annotationdefaults), sets the
#             default property values to use for elements of
#             layout.annotations
#         autosize
#             Determines whether or not a layout width or height that
#             has been left undefined by the user is initialized on
#             each relayout. Note that, regardless of this attribute,
#             an undefined layout width or height is always
#             initialized on the first call to plot.
#         bargap
#             Sets the gap (in plot fraction) between bars of
#             adjacent location coordinates.
#         bargroupgap
#             Sets the gap (in plot fraction) between bars of the
#             same location coordinate.
#         barmode
#             Determines how bars at the same location coordinate are
#             displayed on the graph. With "stack", the bars are
#             stacked on top of one another With "relative", the bars
#             are stacked on top of one another, with negative values
#             below the axis, positive values above With "group", the
#             bars are plotted next to one another centered around
#             the shared location. With "overlay", the bars are
#             plotted over one another, you might need to an
#             "opacity" to see multiple bars.
#         barnorm
#             Sets the normalization for bar traces on the graph.
#             With "fraction", the value of each bar is divided by
#             the sum of all values at that location coordinate.
#             "percent" is the same but multiplied by 100 to show
#             percentages.
#         boxgap
#             Sets the gap (in plot fraction) between boxes of
#             adjacent location coordinates. Has no effect on traces
#             that have "width" set.
#         boxgroupgap
#             Sets the gap (in plot fraction) between boxes of the
#             same location coordinate. Has no effect on traces that
#             have "width" set.
#         boxmode
#             Determines how boxes at the same location coordinate
#             are displayed on the graph. If "group", the boxes are
#             plotted next to one another centered around the shared
#             location. If "overlay", the boxes are plotted over one
#             another, you might need to set "opacity" to see them
#             multiple boxes. Has no effect on traces that have
#             "width" set.
#         calendar
#             Sets the default calendar system to use for
#             interpreting and displaying dates throughout the plot.
#         clickmode
#             Determines the mode of single click interactions.
#             "event" is the default value and emits the
#             `plotly_click` event. In addition this mode emits the
#             `plotly_selected` event in drag modes "lasso" and
#             "select", but with no event data attached (kept for
#             compatibility reasons). The "select" flag enables
#             selecting single data points via click. This mode also
#             supports persistent selections, meaning that pressing
#             Shift while clicking, adds to / subtracts from an
#             existing selection. "select" with `hovermode`: "x" can
#             be confusing, consider explicitly setting `hovermode`:
#             "closest" when using this feature. Selection events are
#             sent accordingly as long as "event" flag is set as
#             well. When the "event" flag is missing, `plotly_click`
#             and `plotly_selected` events are not fired.
#         coloraxis
#             :class:`plotly.graph_objects.layout.Coloraxis` instance
#             or dict with compatible properties
#         colorscale
#             :class:`plotly.graph_objects.layout.Colorscale`
#             instance or dict with compatible properties
#         colorway
#             Sets the default trace colors.
#         datarevision
#             If provided, a changed value tells `Plotly.react` that
#             one or more data arrays has changed. This way you can
#             modify arrays in-place rather than making a complete
#             new copy for an incremental change. If NOT provided,
#             `Plotly.react` assumes that data arrays are being
#             treated as immutable, thus any data array with a
#             different identity from its predecessor contains new
#             data.
#         direction
#             Legacy polar charts are deprecated! Please switch to
#             "polar" subplots. Sets the direction corresponding to
#             positive angles in legacy polar charts.
#         dragmode
#             Determines the mode of drag interactions. "select" and
#             "lasso" apply only to scatter traces with markers or
#             text. "orbit" and "turntable" apply only to 3D scenes.
#         editrevision
#             Controls persistence of user-driven changes in
#             `editable: true` configuration, other than trace names
#             and axis titles. Defaults to `layout.uirevision`.
#         extendfunnelareacolors
#             If `true`, the funnelarea slice colors (whether given
#             by `funnelareacolorway` or inherited from `colorway`)
#             will be extended to three times its original length by
#             first repeating every color 20% lighter then each color
#             20% darker. This is intended to reduce the likelihood
#             of reusing the same color when you have many slices,
#             but you can set `false` to disable. Colors provided in
#             the trace, using `marker.colors`, are never extended.
#         extendpiecolors
#             If `true`, the pie slice colors (whether given by
#             `piecolorway` or inherited from `colorway`) will be
#             extended to three times its original length by first
#             repeating every color 20% lighter then each color 20%
#             darker. This is intended to reduce the likelihood of
#             reusing the same color when you have many slices, but
#             you can set `false` to disable. Colors provided in the
#             trace, using `marker.colors`, are never extended.
#         extendsunburstcolors
#             If `true`, the sunburst slice colors (whether given by
#             `sunburstcolorway` or inherited from `colorway`) will
#             be extended to three times its original length by first
#             repeating every color 20% lighter then each color 20%
#             darker. This is intended to reduce the likelihood of
#             reusing the same color when you have many slices, but
#             you can set `false` to disable. Colors provided in the
#             trace, using `marker.colors`, are never extended.
#         extendtreemapcolors
#             If `true`, the treemap slice colors (whether given by
#             `treemapcolorway` or inherited from `colorway`) will be
#             extended to three times its original length by first
#             repeating every color 20% lighter then each color 20%
#             darker. This is intended to reduce the likelihood of
#             reusing the same color when you have many slices, but
#             you can set `false` to disable. Colors provided in the
#             trace, using `marker.colors`, are never extended.
#         font
#             Sets the global font. Note that fonts used in traces
#             and other layout components inherit from the global
#             font.
#         funnelareacolorway
#             Sets the default funnelarea slice colors. Defaults to
#             the main `colorway` used for trace colors. If you
#             specify a new list here it can still be extended with
#             lighter and darker colors, see
#             `extendfunnelareacolors`.
#         funnelgap
#             Sets the gap (in plot fraction) between bars of
#             adjacent location coordinates.
#         funnelgroupgap
#             Sets the gap (in plot fraction) between bars of the
#             same location coordinate.
#         funnelmode
#             Determines how bars at the same location coordinate are
#             displayed on the graph. With "stack", the bars are
#             stacked on top of one another With "group", the bars
#             are plotted next to one another centered around the
#             shared location. With "overlay", the bars are plotted
#             over one another, you might need to an "opacity" to see
#             multiple bars.
#         geo
#             :class:`plotly.graph_objects.layout.Geo` instance or
#             dict with compatible properties
#         grid
#             :class:`plotly.graph_objects.layout.Grid` instance or
#             dict with compatible properties
#         height
#             Sets the plot's height (in px).
#         hiddenlabels
#             hiddenlabels is the funnelarea & pie chart analog of
#             visible:'legendonly' but it can contain many labels,
#             and can simultaneously hide slices from several
#             pies/funnelarea charts
#         hiddenlabelssrc
#             Sets the source reference on Chart Studio Cloud for
#             hiddenlabels .
#         hidesources
#             Determines whether or not a text link citing the data
#             source is placed at the bottom-right cored of the
#             figure. Has only an effect only on graphs that have
#             been generated via forked graphs from the Chart Studio
#             Cloud (at https://chart-studio.plotly.com or on-
#             premise).
#         hoverdistance
#             Sets the default distance (in pixels) to look for data
#             to add hover labels (-1 means no cutoff, 0 means no
#             looking for data). This is only a real distance for
#             hovering on point-like objects, like scatter points.
#             For area-like objects (bars, scatter fills, etc)
#             hovering is on inside the area and off outside, but
#             these objects will not supersede hover on point-like
#             objects in case of conflict.
#         hoverlabel
#             :class:`plotly.graph_objects.layout.Hoverlabel`
#             instance or dict with compatible properties
#         hovermode
#             Determines the mode of hover interactions. If
#             "closest", a single hoverlabel will appear for the
#             "closest" point within the `hoverdistance`. If "x" (or
#             "y"), multiple hoverlabels will appear for multiple
#             points at the "closest" x- (or y-) coordinate within
#             the `hoverdistance`, with the caveat that no more than
#             one hoverlabel will appear per trace. If *x unified*
#             (or *y unified*), a single hoverlabel will appear
#             multiple points at the closest x- (or y-) coordinate
#             within the `hoverdistance` with the caveat that no more
#             than one hoverlabel will appear per trace. In this
#             mode, spikelines are enabled by default perpendicular
#             to the specified axis. If false, hover interactions are
#             disabled. If `clickmode` includes the "select" flag,
#             `hovermode` defaults to "closest". If `clickmode` lacks
#             the "select" flag, it defaults to "x" or "y" (depending
#             on the trace's `orientation` value) for plots based on
#             cartesian coordinates. For anything else the default
#             value is "closest".
#         images
#             A tuple of :class:`plotly.graph_objects.layout.Image`
#             instances or dicts with compatible properties
#         imagedefaults
#             When used in a template (as
#             layout.template.layout.imagedefaults), sets the default
#             property values to use for elements of layout.images
#         legend
#             :class:`plotly.graph_objects.layout.Legend` instance or
#             dict with compatible properties
#         mapbox
#             :class:`plotly.graph_objects.layout.Mapbox` instance or
#             dict with compatible properties
#         margin
#             :class:`plotly.graph_objects.layout.Margin` instance or
#             dict with compatible properties
#         meta
#             Assigns extra meta information that can be used in
#             various `text` attributes. Attributes such as the
#             graph, axis and colorbar `title.text`, annotation
#             `text` `trace.name` in legend items, `rangeselector`,
#             `updatemenus` and `sliders` `label` text all support
#             `meta`. One can access `meta` fields using template
#             strings: `%{meta[i]}` where `i` is the index of the
#             `meta` item in question. `meta` can also be an object
#             for example `{key: value}` which can be accessed
#             %{meta[key]}.
#         metasrc
#             Sets the source reference on Chart Studio Cloud for
#             meta .
#         modebar
#             :class:`plotly.graph_objects.layout.Modebar` instance
#             or dict with compatible properties
#         newshape
#             :class:`plotly.graph_objects.layout.Newshape` instance
#             or dict with compatible properties
#         orientation
#             Legacy polar charts are deprecated! Please switch to
#             "polar" subplots. Rotates the entire polar by the given
#             angle in legacy polar charts.
#         paper_bgcolor
#             Sets the background color of the paper where the graph
#             is drawn.
#         piecolorway
#             Sets the default pie slice colors. Defaults to the main
#             `colorway` used for trace colors. If you specify a new
#             list here it can still be extended with lighter and
#             darker colors, see `extendpiecolors`.
#         plot_bgcolor
#             Sets the background color of the plotting area in-
#             between x and y axes.
#         polar
#             :class:`plotly.graph_objects.layout.Polar` instance or
#             dict with compatible properties
#         radialaxis
#             :class:`plotly.graph_objects.layout.RadialAxis`
#             instance or dict with compatible properties
#         scene
#             :class:`plotly.graph_objects.layout.Scene` instance or
#             dict with compatible properties
#         selectdirection
#             When `dragmode` is set to "select", this limits the
#             selection of the drag to horizontal, vertical or
#             diagonal. "h" only allows horizontal selection, "v"
#             only vertical, "d" only diagonal and "any" sets no
#             limit.
#         selectionrevision
#             Controls persistence of user-driven changes in selected
#             points from all traces.
#         separators
#             Sets the decimal and thousand separators. For example,
#             *. * puts a '.' before decimals and a space between
#             thousands. In English locales, dflt is ".," but other
#             locales may alter this default.
#         shapes
#             A tuple of :class:`plotly.graph_objects.layout.Shape`
#             instances or dicts with compatible properties
#         shapedefaults
#             When used in a template (as
#             layout.template.layout.shapedefaults), sets the default
#             property values to use for elements of layout.shapes
#         showlegend
#             Determines whether or not a legend is drawn. Default is
#             `true` if there is a trace to show and any of these: a)
#             Two or more traces would by default be shown in the
#             legend. b) One pie trace is shown in the legend. c) One
#             trace is explicitly given with `showlegend: true`.
#         sliders
#             A tuple of :class:`plotly.graph_objects.layout.Slider`
#             instances or dicts with compatible properties
#         sliderdefaults
#             When used in a template (as
#             layout.template.layout.sliderdefaults), sets the
#             default property values to use for elements of
#             layout.sliders
#         spikedistance
#             Sets the default distance (in pixels) to look for data
#             to draw spikelines to (-1 means no cutoff, 0 means no
#             looking for data). As with hoverdistance, distance does
#             not apply to area-like objects. In addition, some
#             objects can be hovered on but will not generate
#             spikelines, such as scatter fills.
#         sunburstcolorway
#             Sets the default sunburst slice colors. Defaults to the
#             main `colorway` used for trace colors. If you specify a
#             new list here it can still be extended with lighter and
#             darker colors, see `extendsunburstcolors`.
#         template
#             Default attributes to be applied to the plot. This
#             should be a dict with format: `{'layout':
#             layoutTemplate, 'data': {trace_type: [traceTemplate,
#             ...], ...}}` where `layoutTemplate` is a dict matching
#             the structure of `figure.layout` and `traceTemplate` is
#             a dict matching the structure of the trace with type
#             `trace_type` (e.g. 'scatter'). Alternatively, this may
#             be specified as an instance of
#             plotly.graph_objs.layout.Template.  Trace templates are
#             applied cyclically to traces of each type. Container
#             arrays (eg `annotations`) have special handling: An
#             object ending in `defaults` (eg `annotationdefaults`)
#             is applied to each array item. But if an item has a
#             `templateitemname` key we look in the template array
#             for an item with matching `name` and apply that
#             instead. If no matching `name` is found we mark the
#             item invisible. Any named template item not referenced
#             is appended to the end of the array, so this can be
#             used to add a watermark annotation or a logo image, for
#             example. To omit one of these items on the plot, make
#             an item with matching `templateitemname` and `visible:
#             false`.
#         ternary
#             :class:`plotly.graph_objects.layout.Ternary` instance
#             or dict with compatible properties
#         title
#             :class:`plotly.graph_objects.layout.Title` instance or
#             dict with compatible properties
#         titlefont
#             Deprecated: Please use layout.title.font instead. Sets
#             the title font. Note that the title's font used to be
#             customized by the now deprecated `titlefont` attribute.
#         transition
#             Sets transition options used during Plotly.react
#             updates.
#         treemapcolorway
#             Sets the default treemap slice colors. Defaults to the
#             main `colorway` used for trace colors. If you specify a
#             new list here it can still be extended with lighter and
#             darker colors, see `extendtreemapcolors`.
#         uirevision
#             Used to allow user interactions with the plot to
#             persist after `Plotly.react` calls that are unaware of
#             these interactions. If `uirevision` is omitted, or if
#             it is given and it changed from the previous
#             `Plotly.react` call, the exact new figure is used. If
#             `uirevision` is truthy and did NOT change, any
#             attribute that has been affected by user interactions
#             and did not receive a different value in the new figure
#             will keep the interaction value. `layout.uirevision`
#             attribute serves as the default for `uirevision`
#             attributes in various sub-containers. For finer control
#             you can set these sub-attributes directly. For example,
#             if your app separately controls the data on the x and y
#             axes you might set `xaxis.uirevision=*time*` and
#             `yaxis.uirevision=*cost*`. Then if only the y data is
#             changed, you can update `yaxis.uirevision=*quantity*`
#             and the y axis range will reset but the x axis range
#             will retain any user-driven zoom.
#         uniformtext
#             :class:`plotly.graph_objects.layout.Uniformtext`
#             instance or dict with compatible properties
#         updatemenus
#             A tuple of
#             :class:`plotly.graph_objects.layout.Updatemenu`
#             instances or dicts with compatible properties
#         updatemenudefaults
#             When used in a template (as
#             layout.template.layout.updatemenudefaults), sets the
#             default property values to use for elements of
#             layout.updatemenus
#         violingap
#             Sets the gap (in plot fraction) between violins of
#             adjacent location coordinates. Has no effect on traces
#             that have "width" set.
#         violingroupgap
#             Sets the gap (in plot fraction) between violins of the
#             same location coordinate. Has no effect on traces that
#             have "width" set.
#         violinmode
#             Determines how violins at the same location coordinate
#             are displayed on the graph. If "group", the violins are
#             plotted next to one another centered around the shared
#             location. If "overlay", the violins are plotted over
#             one another, you might need to set "opacity" to see
#             them multiple violins. Has no effect on traces that
#             have "width" set.
#         waterfallgap
#             Sets the gap (in plot fraction) between bars of
#             adjacent location coordinates.
#         waterfallgroupgap
#             Sets the gap (in plot fraction) between bars of the
#             same location coordinate.
#         waterfallmode
#             Determines how bars at the same location coordinate are
#             displayed on the graph. With "group", the bars are
#             plotted next to one another centered around the shared
#             location. With "overlay", the bars are plotted over one
#             another, you might need to an "opacity" to see multiple
#             bars.
#         width
#             Sets the plot's width (in px).
#         xaxis
#             :class:`plotly.graph_objects.layout.XAxis` instance or
#             dict with compatible properties
#         yaxis
#             :class:`plotly.graph_objects.layout.YAxis` instance or
#             dict with compatible properties

In [83]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=teres_df["Z age"], name='Z'))
fig.add_trace(go.Histogram(x=teres_df["L age"], name='L'))
fig.add_trace(go.Histogram(x=teres_df["B age"], name='B'))

# Overlay both histograms
fig.update_layout(
    
    title=dict(
        text='L. teres age counts Z, L & B',
        x=0.5,
        y=0.95,
        xanchor='center',
        yanchor= 'top',
        font=dict(
            size=16,
            color='#000000'
                )),
    barmode='group',
    bargap=0,
    template='simple_white',
#     paper_bgcolor='#FFFFFF',
#     plot_bgcolor='#FFFFFF',
    width=900, 
    height=500,
    bargroupgap=0
    )
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.update_xaxes(
        showgrid=False, 
        zeroline=False,
        title_text = "age"
        )
fig.update_yaxes(
        showgrid=False, 
        zeroline=False,
        title_text = "count",
        )

fig.show()